BI名企班 谢雅楠

### Thinking

#### 1. 对于同一个带约束的规划问题，LP和MIP哪个运算复杂度高，为什么？
LP研究线性约束条件下线性目标函数的极值问题  
MIP混合整数规划是LP的一种，其中部分的决策变量是整数（不要求全部都是整数）  
因此MIP的复杂度更高。

#### 2. TSP与VRP问题的关系是怎样的？
TSP问题，Travelling Salesman Problem  
一个旅行商想去拜访若干城市，然后回到他的出发地，给定各个城市之间所需的旅行时间后，  
怎样计划他的路线，使得他能对每个城市恰好进行一次访问，而且总时间最短。  

VRP问题，Vehicle Routing Problem  
有N辆车，都从原点出发，每辆车访问一些点后回到原点，  
要求所有的点都要被访问到，求最短的车辆行驶距离或最少需要的车辆数

关系：  
车辆路径问题是旅行商问题的推广。在VRP中，目标是为向不同地点交付货物或服务的车队找到最优路线集。

与TSP类似，VRP也可以用分配给边缘的距离的图来表示。  

如果试图找到一组总距离最小、对车辆没有附加约束的路线，那么最优解决方案是将所有位置分配给一辆车，其余位置空闲。在这种情况下，问题归结为TSP。  
一个更有趣的问题是最小化所有车辆的最长路线距离的长度，或最长旅行时间的路线的消耗时间。VRPs还可以对车辆有额外的限制—例如，对每辆车访问的地点数量或每条路线的长度的下限和上限。


ref.https://zhuanlan.zhihu.com/p/55868067


### Action

In [1]:
import pandas as pd
import numpy as np

#### Step1, 数据加载

In [2]:
data = pd.read_csv("family_data.csv", index_col="family_id")
data

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...
4995,16,1,66,33,18,70,56,46,86,60,4
4996,88,66,20,17,26,54,81,91,59,48,2
4997,32,66,54,17,27,21,74,81,3,7,6


#### Step2，数据预处理
1）计算Perference Cost矩阵 pcost_mat  
2）计算Accounting Cost矩阵 acost_mat  
3）计算每个家庭的人数 FAMILY_SIZE  
4）每个家庭的倾向选择（choice_） DESIRED


In [3]:
## 1. preference cost

# n 家庭成员个数，如果满足choice的需求，需要的penalty
def get_penalty(n, choice):
    penalty = None
    if choice == 0:
        penalty = 0
    if choice == 1:
        penalty = 50
    if choice == 2:
        penalty = 50+9*n
    if choice == 3:
        penalty = 100+9*n
    if choice == 4:
        penalty = 200+9*n
    if choice == 5: 
        penalty = 100+18*n
    if choice == 6:
        penalty = 300+18*n
    if choice == 7: 
        penalty = 300+36*n
    if choice == 8: 
        penalty = 400+36*n
    if choice == 9:
        penalty = 500+(36+199)*n
    if choice > 9:
        penalty = 500+(36+398)*n
    return penalty

In [4]:
N_DAYS = 100
N_FAMILY = 5000
MIN_OCCUPANCY = 125
MAX_OCCUPANCY = 300

## pcost
# 每个家庭在什么时候（day 0-99）访问的penalty
pcost_mat = np.full(shape=(N_FAMILY, 100), fill_value=99999)
for f in range(N_FAMILY):
    f_num = data.loc[f, 'n_people']
#     print(f_num)
    # 第f个家庭，初始化pcost=choice11
    pcost_mat[f,:] = get_penalty(f_num, 10)
    for choice in range(10):
        temp = data.loc[f][choice]
        penalty = get_penalty(f_num, choice)
        pcost_mat[f, temp-1] = penalty

pcost_mat

array([[2236, 2236, 2236, ..., 2236, 2236, 2236],
       [2236, 2236, 2236, ..., 2236, 2236, 2236],
       [1802, 1802, 1802, ..., 1802, 1802,    0],
       ...,
       [3104, 3104,  616, ..., 3104, 3104, 3104],
       [ 390, 2670, 2670, ..., 2670, 2670, 2670],
       [2236, 2236, 2236, ..., 2236, 2236, 2236]])

In [5]:
## acost_mat
# 前一天的参观人数，当天的参观人数
acost_mat = np.zeros(shape=(MAX_OCCUPANCY+1,MAX_OCCUPANCY+1), dtype=np.float64)
for i in range(acost_mat.shape[0]):
    for j in range(acost_mat.shape[1]):
        diff = abs(i-j)
        acost_mat[i,j] = max(0,(i-125) /400 *i **(0.5+diff/50.0))
        
acost_mat

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.16316072e+15, 3.71482922e+15, 3.31477861e+15, ...,
        7.46610759e+00, 8.36716954e+00, 9.37697794e+00],
       [4.79555148e+15, 4.27883100e+15, 3.81778713e+15, ...,
        8.43020770e+00, 7.52185316e+00, 8.43020770e+00],
       [5.52415954e+15, 4.92860244e+15, 4.39725208e+15, ...,
        9.51970597e+00, 8.49339085e+00, 7.57772228e+00]])

In [6]:
## FAMILY_SIZE
FAMILY_SIZE=data['n_people'].values
FAMILY_SIZE

array([4, 4, 3, ..., 6, 5, 4])

In [7]:
## DESIRED
DESIRED = data.values[:,:-1] -1
DESIRED

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]])

#### Step3，使用LP和MIP求解 规划方案
1）先使用LP 对绝大部分家庭进行规划  
2）再使用MIP 对剩余家庭进行规划  
3）汇总两边的结果 => 最终规划方案  

In [8]:
from ortools.linear_solver import pywraplp

In [9]:
## lp 
def solveLP():
    solver= pywraplp.Solver('AssignmentProblem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    x = {}
    candidates = [[] for x in range(N_DAYS)]
    
    for i in range(N_FAMILY):
        for j in DESIRED[i, :]:
            candidates[j].append(i)
            # 决策变量
            x[i,j] = solver.BoolVar('x[%i, %i]' %(i,j))
            
    daily_occupancy = [solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]]) for j in range(N_DAYS)]
    
    
    family_presence = [solver.Sum(x[i,j] for j in DESIRED[i,:]) for i in range(N_FAMILY)]
    
    # 目标函数
    preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] for i in range(N_FAMILY) for j in  DESIRED[i,:]])

    solver.Minimize(preference_cost)
    
    #认为约束
    for j in range(N_DAYS-1):
        solver.Add(daily_occupancy[j] - daily_occupancy[j+1] <= 25)
        solver.Add(daily_occupancy[j+1] - daily_occupancy[j] <= 25)
    
    for i in range(N_FAMILY):
        solver.Add(family_presence[i] == 1)
    
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j] >= MIN_OCCUPANCY)
        solver.Add(daily_occupancy[j] <= MAX_OCCUPANCY)
        
    result = solver.Solve()
    
    
    ## result
    temp = [(i,j, x[i,j].solution_value()) for i in range(N_FAMILY) for j in DESIRED[i,:] if x[i,j].solution_value() >0]
    
    df = pd.DataFrame(temp, columns=['family_id', 'day', 'result'])
    
    return df



In [10]:
%%time
result = solveLP()
result

CPU times: user 10.5 s, sys: 72.1 ms, total: 10.6 s
Wall time: 10.6 s


,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5067,4995,15,1.0
5068,4996,87,1.0
5069,4997,31,1.0
5070,4998,91,1.0


In [11]:
THRS = 0.999
assigned_df = result[result.result > THRS]
assigned_df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5067,4995,15,1.0
5068,4996,87,1.0
5069,4997,31,1.0
5070,4998,91,1.0


In [12]:
# 没有安排的家庭
unassigned_df = result[(result.result < THRS) & (result.result > 1-THRS)] 
unassigned_df

,family_id,day,result
16,16,45,0.750000
17,16,49,0.250000
60,59,38,0.875000
61,59,14,0.125000
190,188,9,0.666667
...,...,...,...
4956,4886,98,0.750000
4984,4914,38,0.500000
4985,4914,43,0.500000
5032,4961,53,0.750000


In [13]:
### 对剩余家庭进行规划
assigned_df['family_size'] = FAMILY_SIZE[assigned_df.family_id]
assigned_df

<ipython-input-13-9bdd94f460fb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assigned_df['family_size'] = FAMILY_SIZE[assigned_df.family_id]


,family_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5067,4995,15,1.0,4
5068,4996,87,1.0,2
5069,4997,31,1.0,6
5070,4998,91,1.0,5


In [14]:
occupancy = assigned_df.groupby('day').family_size.sum().values
occupancy

array([275, 271, 299, 299, 270, 242, 223, 246, 269, 298, 290, 300, 275,
       250, 238, 272, 298, 298, 274, 248, 223, 241, 259, 286, 298, 298,
       273, 249, 239, 255, 278, 281, 252, 228, 209, 184, 202, 233, 253,
       231, 209, 183, 204, 225, 247, 278, 251, 227, 204, 198, 223, 248,
       260, 228, 208, 185, 173, 196, 220, 198, 162, 148, 119, 114, 149,
       175, 189, 160, 136, 119, 125, 135, 158, 185, 158, 127, 125, 125,
       142, 167, 185, 167, 141, 120, 128, 155, 174, 200, 179, 152, 130,
       122, 136, 158, 181, 157, 134, 125, 122, 124])

In [15]:
min_occupancy = np.array([max(0, MIN_OCCUPANCY - x) for x in occupancy])
min_occupancy

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6, 11,  0,  0,  0,  0,
        0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  0,
        0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  3,  1])

In [16]:
max_occupancy = np.array([ MAX_OCCUPANCY - x for x in occupancy])
max_occupancy

array([ 25,  29,   1,   1,  30,  58,  77,  54,  31,   2,  10,   0,  25,
        50,  62,  28,   2,   2,  26,  52,  77,  59,  41,  14,   2,   2,
        27,  51,  61,  45,  22,  19,  48,  72,  91, 116,  98,  67,  47,
        69,  91, 117,  96,  75,  53,  22,  49,  73,  96, 102,  77,  52,
        40,  72,  92, 115, 127, 104,  80, 102, 138, 152, 181, 186, 151,
       125, 111, 140, 164, 181, 175, 165, 142, 115, 142, 173, 175, 175,
       158, 133, 115, 133, 159, 180, 172, 145, 126, 100, 121, 148, 170,
       178, 164, 142, 119, 143, 166, 175, 178, 176])

In [17]:
def solveIP(families, min_occupancy, max_occupancy):
    solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    
    n_families = len(families)
    
    x = {}
    candidates = [[] for x in range(N_DAYS)]
    
    for i in families:
        for j in DESIRED[i, :]:
            candidates[j].append(i)
            # 决策变量
            x[i,j] = solver.BoolVar('x[%i, %i]' %(i,j))
            
    daily_occupancy = [solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]]) for j in range(N_DAYS)]
    
    
    family_presence = [solver.Sum(x[i,j] for j in DESIRED[i,:]) for i in families]
    
    # 目标函数
    preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] for i in families for j in  DESIRED[i,:]])

    solver.Minimize(preference_cost)
    
    #认为约束
    for j in range(N_DAYS-1):
        solver.Add(daily_occupancy[j] - daily_occupancy[j+1] <= 25)
        solver.Add(daily_occupancy[j+1] - daily_occupancy[j] <= 25)
    
    for i in range(n_families):
        solver.Add(family_presence[i] == 1)
    
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j] >= min_occupancy[j])
        solver.Add(daily_occupancy[j] <= max_occupancy[j])
        
    result = solver.Solve()
    
    ## result
    temp = [(i,j) for i in families for j in DESIRED[i,:] if x[i,j].solution_value() >0]
    
    df = pd.DataFrame(temp, columns=['family_id', 'day'])
    
    return df

In [18]:
unassigned = unassigned_df.family_id.unique()

result1 = solveIP(unassigned, min_occupancy, max_occupancy)

In [19]:
result1

,family_id,day
0,16,45
1,59,38
2,188,19
3,240,32
4,455,0
...,...,...
59,4619,67
60,4870,69
61,4886,98
62,4914,38


In [20]:
## 汇总结果
df = pd.concat((assigned_df[['family_id', 'day']], result1)).sort_values('family_id')
df

,family_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
5067,4995,15
5068,4996,87
5069,4997,31
5070,4998,91


#### Step4, 结果评估
按照evaluation标准，计算
Score = preference cost + accounting penalty


In [21]:
def pcost(prediction):
    daily_occupancy = np.zeros(N_DAYS+1, dtype=np.int64)
    penalty = 0
    for (i,p) in enumerate(prediction):
        n = FAMILY_SIZE[i]
        penalty += pcost_mat[i,p]
        daily_occupancy += n
    return penalty, daily_occupancy

def acost(daily_occupancy):
    accounting_cost = 0
    number_outofrange = 0
    daily_occupancy[-1] = daily_occupancy[-2]
    
    for day in range(N_DAYS):
        n_p1 = daily_occupancy[day+1]
        n = daily_occupancy[day]
        
        number_outofrange += (n>MAX_OCCUPANCY) or (n<MIN_OCCUPANCY)
        accounting_cost += acost_mat[n, n_p1]
    return accounting_cost, num_outofrange

def cost_function(prediction):
    penalty, daily_occupancy = pcost(prediction)
    accounting_cost, num_outofrange = acost(daily_occupancy)
    final_score = penalty + accounting_cost + num_outofrange*99999999
    return final_score

In [23]:
prediction = df.day.values
print(cost_function(prediction))

76896.4367999826


In [25]:
def save_result(pred, filename):
    result = pd.DataFrame(range(N_FAMILY), columns=['family_id'])
    result['assigned_day'] = pred+1
    result.to_csv(filename, index=False)
    return result
result = save_result(prediction, 'submission.csv')

#### Step5，方案优化
通过更换family_id的选择，查找更好的score
每次更换后，都对方案进行评估，选择更小的score方案
